In [ ]:
!pip install bs4
!pip install requests

In [92]:
# import the dependencies

from bs4 import BeautifulSoup
import requests
import pandas as pd
import nltk

nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [93]:
# read the csv file
df = pd.read_csv("input_file.csv")

# assign all the urls in a single variable
urls = df['url']

#index variable
link = 0

In [94]:
column_names = [
    'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
    'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
    'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
    'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
    'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'
]

# allocating the outplut variables column to 0 initially
for name in column_names:
  df[name] = 0

In [95]:
# stop words
stop_words = []

with open("StopWords_Auditor.txt", "r") as f:
    temp = f.read()

stop_words = temp.split()


with open("StopWords_Currencies.txt", "r", encoding="ISO-8859-1") as f:
    temp = f.read()
    stop_words += temp.split()
    stop_words = [i for i in stop_words if i != "|"]



with open("StopWords_DatesandNumbers.txt", "r") as f:
    temp = f.read()
    stop_words += temp.split()
    stop_words = [i for i in stop_words if i != "|"]


with open("StopWords_Generic.txt", "r") as f:
    temp = f.read()
    stop_words += temp.split()
    stop_words = [i for i in stop_words if i != "|"]

with open("StopWords_GenericLong.txt", "r") as f:
    temp = f.read()
    stop_words += temp.split()
    stop_words = [i for i in stop_words if i != "|"]

with open("StopWords_Geographic.txt", "r") as f:
    temp = f.read()
    stop_words += temp.split()
    stop_words = [i for i in stop_words if i != "|"]


with open("StopWords_Names.txt", "r") as f:
    temp = f.read()
    stop_words += temp.split()
    stop_words = [i for i in stop_words if i != "|"]

# successfully all the stop words are collected in a single list


In [96]:
def text_analysis(text, stop_words):
  from nltk.tokenize import sent_tokenize

  #tokenizing the sentence
  token_sent = sent_tokenize(text)

  from nltk import tokenize

  #tokenizing the words
  token_words = tokenize.word_tokenize(text)

  # cleaning words - removing the stop words from token words

  words_after_cleaning = []

  stop_words = [i.lower() for i in stop_words]

  for word in token_words:
    if word not in stop_words:
      words_after_cleaning += [word]

  # cleaning sentences - removing stop words from each sentence

  sentence_after_cleaning = []

  for word in token_sent:
    temp = ""
    sent = word.split()
    for w in sent:
      if w not in stop_words:
        temp += " " + w
    sentence_after_cleaning.append(temp)

  #positive words

  with open("positive_words.txt", "r") as f:
      positive_list = f.read()

  positive_list = positive_list.split()

  #negative words

  with open("negative-words.txt", "r", encoding="ISO-8859-1") as f:
      neg_list = f.read()

  negative_list = neg_list.split()

  #positive score - number of positive words present in the token words

  positive_set = set(positive_list)

  positive_score = 0

  token_words = [i.lower() for i in token_words]

  for word in positive_set:
    if word.lower() in token_words:
      positive_score += 1

  #negative score -  number of negative words present in the token words

  negative_set = set(negative_list)

  negative_score = 0

  for word in negative_set:
      if word.lower() in token_words:
        negative_score += 1

  #polarity score

  polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)

  total_words_after_cleaning = 0


  # removing the punctuations

  punctuations = [".", ";", ":", ",", "-", "`", "~", "!", "?", "'", "’"]
  words_after_cleaning = [i for i in words_after_cleaning if i not in punctuations]

  total_words_after_cleaning = len(words_after_cleaning)

  # subjectivity score

  subjectivity_score = (positive_score - negative_score) / ((total_words_after_cleaning) + 0.000001)


  # syllable count in a word

  import nltk
  from nltk.tokenize import SyllableTokenizer

  # Download the nltk data needed for the syllable tokenizer

  def count_syllables_nltk(word):
      syllable_tokenizer = SyllableTokenizer()
      syllables = syllable_tokenizer.tokenize(word)
      return len(syllables)

  def count_syllables_in_sentence_nltk(sentence):
      words = nltk.word_tokenize(sentence)
      total_syllables = sum(count_syllables_nltk(word) for word in words)
      return total_syllables

  def syllable_count(word):
    count = count_syllables_nltk(word)

    # handle some exceptions like words ending with "es","ed" by not counting them as a syllable.'
    if len(word) > 2:
      if word[-2] + word[-1] == "ed" or word[-2] + word[-1] == "es":
        count -= 1

      # a silent 'e' at the end of a word doesn't contribute to an additional syllable.
      if word[-1] == "e":
        count -= 1

    return count


  complex_words = [i for i in words_after_cleaning if syllable_count(i) > 2]

  # average sentence length

  average_sentence_length = len(words_after_cleaning) / len(sentence_after_cleaning)

  # complex word percentage

  complex_word_percent = (len(complex_words) / len(words_after_cleaning)) * 100

  # fog index

  fog_index = (average_sentence_length + complex_word_percent) * 0.4

  #average number of word per sentence

  average_number_words_per_sentence = len(token_words) / len(token_sent)

  # complex word count

  complex_word_count = len(complex_words)

  # word count

  word_count = len(words_after_cleaning)

  # syllable count per word

  count_s = 0
  for i in token_words:
    count_s += syllable_count(i)

  syllable_count_per_word = count_s / len(token_words)

  #pronoun count

  pronouns = ["I", "we", "ours", "my", "us"]

  pronoun_count = 0

  for pronoun in pronouns:
    for word in token_words:
      if word == pronoun:
        pronoun_count += 1

  # average word length

  char_length = 0

  for word in token_words:
    char_length += len(word)

  average_word_length = char_length / len(token_words)

  return positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,complex_word_percent,fog_index,average_number_words_per_sentence,complex_word_count,word_count,syllable_count_per_word,pronoun_count,average_word_length

  # print(f"text \n {token_words} \n\n positive score {positive_score} \n negative score {negative_score}")

in try block,
  most of the of the article created inside the div of class name = "td-post-content tagdiv-type" where the main contents are present in the para tags so extracted that alone

in except block,
  some of the article starts with the images so in that case the contents are present inside the div of class name "td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type"
  so the extracted the para tag contents alone

if a link doesnt comes under either of the cases it will be considered the link doesnt working

  

In [97]:
def scrap_article_from_url(url):

  global link

  error_links = []

  status = False

  # url = "https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-2/"

  page = requests.get(url)

  soup = BeautifulSoup(page.text, "html")

  title = soup.title.text

  # after getting the entire information from the page get only the para tag contents

  p_tags = ""

  try:
    div_tags = soup.find_all('div', class_ = "td-post-content tagdiv-type")

    p_tags = div_tags[0].find_all('p')

    status = True

  except IndexError:
    try:
      div_tags = soup.find_all('div', class_ = "td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type")
      p_tags = div_tags[0].find_all('div', class_ = "tdb-block-inner td-fix-index")
      # print(div_tags)

      p_tags = div_tags[0].find_all('p')
      # print(p_tags)

      status = True
    except:
      # error link - index of link that doesnt working properly
      error_links.append(link)

      status = False

  # len(div_tags)

  whole_content = ""


  for i in p_tags:
    whole_content += i.text

  current_index = link
  link += 1

  print(current_index)




  return status, whole_content, current_index


In [ ]:
# main programs starts here
#index initially 0
link = 0

# loop throgh the each url
for url in urls:
  # text - article content
  # index - current index
  # status - is it ready to analyse or not
  status, text, index = scrap_article_from_url(url)

  # if the status is true then url is working properly
  if status == True:
    # result - contains the tuple of output variables
    result = text_analysis(text, stop_words)
    result = list(result) # type casted to list

    # assign the values to their respective columns
    for i in range(len(column_names)):
      df.loc[index, column_names[i]] = result[i]


df.to_csv("final_output.csv", index = False)

'''
 id : ["blackassign0036", "blackassign0049"]
 the above mentioned id and its respective url doesnt working so their entries remains zero(0)
'''
